<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [30]:
from IPython.display import SVG, display, HTML

In [31]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [32]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [33]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  new_anns60k.csv  new_anns60k.csv.1  new_df_anns60k.csv  sample_data
MyDrive
.shortcut-targets-by-id
.file-revisions-by-id
.Trash


In [34]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [35]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-23 08:38:49--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-23 08:38:50 ERROR 404: Not Found.

--2021-05-23 08:38:50--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1192455 (1.1M) [text/plain]
Saving to: ‘new_anns60k.csv.2’

new_anns60k.csv.2   100%[===================>]   1.14M  7.14MB/s    in 0.2s    

20

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [36]:
# with open('coref_df_anns60k.csv') as csvfile:
with open('new_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
      if len(new_row) > 9:
        new_row = new_row[:9]
      writer.writerow(new_row)


In [37]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

Проверим данные

In [38]:
anns_df

,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER
0,обсуждаем,1.0,0.0,0.0,0.0,0.0,обсуждаем,0.0,0.0
1,препарат,1.0,0.0,0.0,1.0,0.0,обсуждаем,0.0,0.0
2,рисперидон,1.0,0.0,0.0,2.0,0.0,препарат,1.0,2.0
3,рисполепт,1.0,0.0,0.0,3.0,0.0,рисперидон,1.0,3.0
4,сперидан,1.0,0.0,0.0,4.0,0.0,рисполепт,1.0,4.0
...,...,...,...,...,...,...,...,...,...
15405,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15406,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15407,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15408,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [39]:
import numpy as np

In [40]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
2,рисперидон,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,[-0.17725945 0.94834274 0.90867615 0.850609...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,0
3,рисполепт,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,[ 0.15259643 1.0673474 0.84192747 -0.077515...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,0
4,сперидан,[ 0.08634768 0.893356 0.55127096 -0.065075...,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.02878256 0.29778534 0.18375699 -0.021691...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26493,таможенные,[ 0.73242044 -0.24206167 -0.6556247 -0.476650...,0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26494,пошлины,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0
26495,как,[ 1.0349948e+00 7.3793733e-01 -3.8374668e-01 ...,0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0


In [41]:
cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

0.8436395154681279

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [42]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]

Добавим размеченные данные, и посмотрим, что получилось.

In [43]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [44]:
df = cor_df.iloc[:15000,:]

In [45]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,"[-0.208233938, 0.996180058, -0.279894114, -0.9...",0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0.0,0.0
1,препарат,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0.0,0.0
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,эта,"[0.46855617, 0.33405855, 0.19523463, -0.341421...",0,16,3,0,10,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,"[0.3625939, 0.13204741, -0.02575476, -0.517570...","[0.0185163, 0.03019085, 0.00883435, 0.01005786...","[0.46855617, 0.33405855, 0.19523463, -0.341421...",0.0,0.0
14996,вакцина,"[0.3625939, 0.13204741, -0.02575476, -0.517570...",0,16,3,0,11,inan,nomn,femn,NaN,NOUN,nsubj,аукнется,NaN,NaN,NaN,VERB,"[0.4076383, 0.26426148, 0.31928974, 0.1378843,...","[0.0185163, 0.03019085, 0.00883435, 0.01005786...","[0.49104294, 0.533049, -0.17000696, 0.3062497,...",0.0,0.0
14997,аукнется,"[0.4076383, 0.26426148, 0.31928974, 0.1378843,...",0,16,3,0,12,NaN,NaN,NaN,3per,VERB,csubj,неизвестно,NaN,NaN,NaN,ADVB,"[0.49159753, 0.09829157, 0.20229428, -0.199118...","[0.0185163, 0.03019085, 0.00883435, 0.01005786...","[0.22764659, 0.2621964, 0.24174161, 0.01494996...",0.0,0.0
14998,на,"[0.76294684, 0.88684666, 0.41325688, 0.3235328...",0,16,3,0,13,NaN,NaN,NaN,NaN,PREP,case,организме,inan,loct,masc,NOUN,"[0.3846327, 0.56424165, 0.09776899, 0.7855562,...","[0.0185163, 0.03019085, 0.00883435, 0.01005786...","[0.76294684, 0.88684666, 0.41325688, 0.3235328...",0.0,0.0


Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [46]:
len(df.loc[df['NER'] == 1])

576

Из них местоимений (прономинальные анафоры).

In [47]:
len(df.loc[df['NER'] == 3])

113

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [48]:
len(df.loc[df['NER'] == 2])

214

In [49]:
len(df.loc[df['NER'].isin([1,2,3])])

903

In [50]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [51]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
21,лекарством,"[0.598588, -0.07832722, 0.23088205, -0.3704158...",0,1,1,1,10,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,"[1.3479745, 0.34477097, 0.20343542, 0.10220611...","[0.05498165, -0.00549405, 0.01865345, -0.03142...","[0.42120528, 0.21546456, 0.00441866, -0.240917...",2.0,0.0
142,рисполепта,"[0.20826544, 0.5118653, 0.6703367, -0.22502856...",0,1,1,13,7,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,"[0.5131939, 0.9708054, 0.53676724, 0.09788364,...","[0.11585727, 0.1283627, 0.02299715, -0.0095005...","[0.28126505, 0.3790152, 0.5397224, -0.09368, -...",1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14737,капецитабин,"[0.342338204, 1.20741296, 0.623081028, 0.04668...",1,15,17,10,1760,anim,nomn,masc,NaN,NOUN,obj,закончили,NaN,NaN,NaN,VERB,"[0.47268036, 0.591915309, 0.0497007072, 9.5443...","[0.03567506, 0.05356697, -0.02985729, 0.017830...","[0.22042012, 0.68094474, 0.0970263, 0.13410695...",1.0,2215.0
14739,пить,"[0.32367313, 1.055545, 0.06944968, 0.37159777,...",1,15,17,10,1762,NaN,NaN,NaN,NaN,INFN,xcomp,закончили,NaN,NaN,NaN,VERB,"[0.47268036, 0.591915309, 0.0497007072, 9.5443...","[0.03567506, 0.05356697, -0.02985729, 0.017830...","[0.22042012, 0.68094474, 0.0970263, 0.13410695...",1.0,2217.0
14786,чем,"[0.02129029, 0.3361394, 0.37177402, -0.1613324...",1,15,18,0,1809,NaN,NaN,NaN,NaN,CONJ,fixed,тогда,NaN,NaN,NaN,ADVB,"[0.63305646, 0.76141953, 0.38590235, -0.319505...","[0.02524, 0.01563716, -0.04932618, -0.00945873...","[0.02129029, 0.3361394, 0.37177402, -0.1613324...",1.0,2264.0
14817,данный,"[0.20950672, 1.1722854, -0.22494371, 0.0924339...",1,15,18,4,1840,inan,accs,masc,NaN,ADJF,amod,момент,inan,accs,masc,NOUN,"[0.13350825, 1.0182619, -0.18650436, 0.4104124...","[0.0310304258, 0.0191320945, 0.027789766, -0.0...","[0.4862268, 1.029566, 0.09415659, 0.20798339, ...",1.0,2295.0


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [52]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
576,зелдокс,"[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.03948287, 0.24082112, 0.22588146, -0.017884...","[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",1.0,2.0
1870,оланзапин,"[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01915067, 0.0875298, 0.09080387, 0.00378973...","[-0.082683861, 1.2017089, 0.67815673, -0.22599...",1.0,2.0
5555,галоперидол,"[-0.03019532, 1.2394575, 0.18222564, -0.215572...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[-0.03747297, 0.5554195, 0.15133946, -0.011017...","[-0.10372739, 1.0939001, 0.5454509, 0.3175185,...",1.0,2.0
6200,тиаприд,"[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.10855111, 0.28903672, 0.2550475, -0.0276597...","[0.27581334, 0.80696917, 1.0271364, -0.1037090...",1.0,3.0
8450,сероквель,"[0.087633356, 0.93445277, 0.69432694, 0.298702...",0,6,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[-0.04207938, 0.36281538, 0.25962716, 0.010672...","[0.087633356, 0.93445277, 0.69432694, 0.298702...",1.0,2.0
8541,день,"[0.11187396, 1.2210127, -0.88301444, -0.466371...",0,7,0,0,2,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,"[0.11187396, 1.2210127, -0.88301444, -0.466371...","[0.10426845, 0.58591, -0.40001512, -0.31153095...","[0.3036255, 0.7870125, -0.33815098, -0.5249253...",0.0,0.0
11117,500,"[0.34515736, 0.99082905, 1.1647757, 0.13400415...",0,8,0,0,2,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,"[0.0851192921, 0.789108872, 0.923557222, -0.23...","[0.00030258, -0.03585999, -0.01680361, -0.0245...",[0.0],0.0,0.0
11787,капецитабин,"[0.342338204, 1.20741296, 0.623081028, 0.04668...",0,10,0,0,2,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,"[0.342338204, 1.20741296, 0.623081028, 0.04668...","[0.00206052, -0.05985366, 0.06623499, -0.06572...","[-0.0034975, 0.14409654, 0.24720684, -0.020498...",0.0,0.0
11806,иресса,"[0.403851897, 0.462512106, 0.103562765, -0.530...",0,11,0,0,2,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,"[0.235153243, -0.000665825617, 0.172356203, 0....","[0.07661632, 0.08160812, 0.05639331, 0.0203737...","[0.12400526, 0.19563669, 0.21844696, -0.525781...",0.0,0.0


In [53]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [54]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [55]:
df.TOKEN_VECT[710]

array([nan])

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [56]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [57]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [58]:
df.TOKEN[1]

'препарат'

In [59]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на данные.

In [60]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,0.478969,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0.514440,0.835993,0.605408,0.0,0.0
1,препарат,0.525897,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0.514440,0.835993,0.605408,0.0,0.0
2,рисперидон,0.779845,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,0.878739,0.835993,0.808772,1.0,2.0
3,рисполепт,0.797031,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,0.554854,0.835993,0.830899,1.0,3.0
4,сперидан,0.760111,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,0.602089,0.835993,0.805134,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,эта,0.411351,0,16,3,0,10,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,0.465146,0.449984,0.398606,0.0,0.0
14996,вакцина,0.506801,0,16,3,0,11,inan,nomn,femn,NaN,NOUN,nsubj,аукнется,NaN,NaN,NaN,VERB,0.617590,0.449984,0.714478,0.0,0.0
14997,аукнется,0.544597,0,16,3,0,12,NaN,NaN,NaN,3per,VERB,csubj,неизвестно,NaN,NaN,NaN,ADVB,0.561118,0.449984,0.608900,0.0,0.0
14998,на,0.450034,0,16,3,0,13,NaN,NaN,NaN,NaN,PREP,case,организме,inan,loct,masc,NOUN,0.493778,0.449984,0.418060,0.0,0.0


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [69]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [86]:
topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)].iloc[[2]]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
13,практически,0.569599,0,1,1,1,2,NaN,NaN,NaN,NaN,ADVB,advmod,нет,NaN,NaN,NaN,PRED,0.520258,0.518605,0.660347,0.0,0.0


In [ ]:
Мы

In [ ]:
df.loc[df['SENT_NUM'] != 0.0]

In [65]:
for i in range(len(df)):
  current_raw = df.iloc[[i]].loc[df['NER'] != 0.0]
  print(current_raw)
  if i == 3:
    break

Empty DataFrame
Columns: [TOKEN, TOKEN_VECT, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, HEAD_VECT, SENT_VECT, HEAD_CHILDS_VECT, NER, COREFERENCE_CLUSTER]
Index: []
Empty DataFrame
Columns: [TOKEN, TOKEN_VECT, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, HEAD_VECT, SENT_VECT, HEAD_CHILDS_VECT, NER, COREFERENCE_CLUSTER]
Index: []
        TOKEN  TOKEN_VECT  ...  NER  COREFERENCE_CLUSTER
2  рисперидон    0.779845  ...  1.0                  2.0

[1 rows x 23 columns]
       TOKEN  TOKEN_VECT  IS_ANSWER  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
3  рисполепт    0.797031          0  ...          0.830899  1.0                  3.0

[1 rows x 23 columns]


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [1404]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [1405]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [1406]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'привился', 'прививки', 'решать', 'неизвестно', 'быстро', 'проводились',
        'испытания', 'аукнется', 'вакцина', 'организме'],
       dtype='object', length=2519),
 'HEAD_ANIMACY': Index(['inan', 'ani

In [1407]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [1408]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,0.779845,0,1,0,0,2,1,0,0,-1,1,2,1,0,0,0,1,0.878739,0.835993,0.808772,1.0,2.0
3,рисполепт,0.797031,0,1,0,0,3,0,1,1,-1,1,3,2,1,0,0,1,0.554854,0.835993,0.830899,1.0,3.0
4,сперидан,0.760111,0,1,0,0,4,-1,-1,0,-1,2,4,3,0,1,1,1,0.602089,0.835993,0.805134,1.0,4.0
21,лекарством,0.456023,0,1,1,1,10,0,5,2,-1,1,4,7,-1,-1,-1,3,0.520258,0.518605,0.660347,2.0,0.0
142,рисполепта,0.731182,0,1,1,13,7,0,0,1,-1,1,21,69,0,2,1,1,0.471137,0.639488,0.762393,1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12726,приходили,0.534609,0,15,0,10,1,-1,-1,-1,-1,0,0,2307,-1,-1,-1,0,0.510058,0.492714,0.630305,1.0,204.0
12785,завершению,0.453896,0,15,0,16,1,0,2,2,-1,1,10,1851,-1,-1,-1,0,0.390514,0.437342,0.503593,1.0,263.0
12854,к,0.415880,0,15,0,22,3,-1,-1,-1,-1,7,9,2332,1,2,1,1,0.626481,0.641312,0.428054,1.0,332.0
12876,стадию,0.542401,0,15,0,25,7,0,3,1,-1,1,1,1777,-1,-1,-1,0,0.475252,0.694040,0.622309,2.0,0.0


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [1409]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [1410]:
train = train.reset_index()

In [1411]:
# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
# train = train.drop(['TOKEN', 'index'], axis=1)

In [1412]:
test = test.reset_index()

In [1413]:
test.head(5)

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,3673,препараты,0.468716,1,3,22,1,1789,0,0,0,-1,1,1,739,-1,-1,-1,5,0.658265,0.673832,0.677477,2.0,1795.0
1,2256,препаратов,0.464472,1,3,5,12,372,0,1,0,-1,1,21,569,0,5,1,1,0.420347,0.502091,0.589498,2.0,388.0
2,9144,химии,0.556986,1,7,1,18,241,0,1,1,-1,1,10,1699,-1,0,2,8,0.465183,0.564593,0.646650,2.0,0.0
3,7336,нейролептиками,0.589178,1,5,35,3,1134,0,5,0,-1,1,21,1459,0,2,1,1,0.418195,0.493534,0.665142,2.0,0.0
4,6526,препарат,0.525897,1,5,16,0,324,0,0,0,-1,1,1,1280,-1,-1,-1,0,0.583459,0.713328,0.764346,2.0,319.0


In [1414]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [1415]:
# test.loc[test['NER'] != 0]

In [1416]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [1417]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [1418]:
X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [1419]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [1420]:
X

,TOKEN_VECT,TOKEN_NUM,GENDER,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,0.519055,221,0,5,0.743045,3.0,218.0
1,0.584090,84,-1,-1,0.615114,2.0,0.0
2,0.454605,35,-1,16,0.525582,2.0,0.0
3,0.457997,62,-1,11,0.600316,2.0,6.0
4,0.471790,1660,1,5,0.739101,3.0,1657.0
...,...,...,...,...,...,...,...
226,0.476860,1568,1,5,0.616808,3.0,1582.0
227,0.525897,320,0,1,0.641136,2.0,663.0
228,0.468716,732,0,1,0.435643,2.0,0.0
229,0.525897,2721,0,1,0.703564,2.0,2711.0


In [1421]:
X_test.head(10)

,TOKEN_VECT,TOKEN_NUM,GENDER,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,0.468716,1789,0,1,0.673832,2.0,1795.0
1,0.464472,372,0,1,0.502091,2.0,388.0
2,0.556986,241,1,1,0.564593,2.0,0.0
3,0.589178,1134,0,1,0.493534,2.0,0.0
4,0.525897,324,0,1,0.713328,2.0,319.0
5,0.406463,72,2,1,0.584244,2.0,0.0
6,0.459385,1032,0,1,0.497320,2.0,1081.0
7,0.505452,512,0,5,0.484634,3.0,507.0
8,0.525897,265,0,1,0.625485,2.0,287.0
9,0.525897,3113,0,1,0.519936,2.0,3125.0


In [1422]:
y

0       218.0
1         0.0
2         0.0
3         6.0
4      1657.0
        ...  
226    1582.0
227     663.0
228       0.0
229    2711.0
230       0.0
Name: COREFERENCE_CLUSTER, Length: 231, dtype: float64

Обучаем модели.

In [1423]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.603448275862069

In [1424]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6379310344827587

In [1425]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.5344827586206896

In [1426]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)

1.0

In [1427]:
from sklearn import svm

In [1428]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.39655172413793105

In [1429]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=7, weights='distance',algorithm='ball_tree',leaf_size=40, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.6724137931034483

In [1430]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [1431]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [1432]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [1433]:
# my_eval(list(naive_pred), list(y_test))

In [1434]:
my_eval(list(pred_knn), list(y_test))

17  true positive
17  false positive
22  true negative
2  false negative
---------
0.5  presicion
0.8947368421052632  recall
0.6415094339622641  f-score


In [1435]:
my_eval(list(pred_entropy), list(y_test))

14  true positive
20  false positive
23  true negative
1  false negative
---------
0.4117647058823529  presicion
0.9333333333333333  recall
0.5714285714285713  f-score


In [1436]:
my_eval(list(pred), list(y_test))

12  true positive
22  false positive
23  true negative
1  false negative
---------
0.35294117647058826  presicion
0.9230769230769231  recall
0.5106382978723405  f-score


In [1437]:
my_eval(list(pred_randf), list(y_test))

8  true positive
26  false positive
23  true negative
1  false negative
---------
0.23529411764705882  presicion
0.8888888888888888  recall
0.3720930232558139  f-score


In [1438]:
my_eval(list(pred_regr), list(y_test))

2  true positive
56  false positive
0  true negative
0  false negative
---------
0.034482758620689655  presicion
1.0  recall
0.06666666666666667  f-score


In [1439]:
my_eval(list(pred_svm), list(y_test))

0  true positive
12  false positive
23  true negative
23  false negative
---------
0.0  presicion
0.0  recall
0  f-score
